# $\mathbf{PREDICTING}$ $\mathbf{PRIMARY}$ $\mathbf{GITHUB}$ $\mathbf{REPOSITORY}$ $\mathbf{LANGUAGE}$ $\mathbf{BASED}$ $\mathbf{ON}$ $\mathbf{README}$ $\mathbf{TEXT}$
## *Examining the top 1000 repos featuring the word 'cat': can we determine which words or phrases in a readme are indicative of one programming language over another?*
### Prepared for The Cat Fancier's Society of Greater Githubsville
$\rightarrow$Notebook and accompanying presentation by Lochlyn Laskowski, Gregory Maggard, and Ryan Miller | Github Data Scientist Team | 7/26/2022

# $\bigstar$ Executive Summary {RYAN - at end}

### Question: 
### Actions: 
### Conclusions:  
### Recommendations:

# $\bigstar$ Libraries Requied

Common DS Libraries

In [1]:
# For data analysis:
import numpy as np
import pandas as pd

# Graphs/Visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# For Statistics and Hypothesis Testing:
import scipy.stats as stats

# NLP tools
import nltk
from wordcloud import WordCloud
import re

# Web Scraping Tools
import requests
from bs4 import BeautifulSoup as bs
import json

# For Classification Modeling:
from itertools import combinations
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

/opt/homebrew/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Internally Created Helper Functions

In [2]:
# For acquiring the dataset from the Cal Tech Database:
from acquire import json_to_df

# For initiual preparing and cleaning of the dataset (text)
from prepare import df_cleaner

# and seperating into new df, X_train_exp, X_train, y_train, X_validate, y_validate, X_test, y_test:
import wrangle import cat_wrangle

# For comparing Classification Algorithm Outcomes
from matrix_result import matrix_result_2d

Others

In [3]:
#Ignore Warnings:
import warnings
warnings.filterwarnings('ignore')

# reloads import files each time a cell is ran (makes your life easier)
%load_ext autoreload
%autoreload 2

# Shows all columns of a dataframe
pd.set_option('display.max_columns', None)

# $\bigstar$ Data Acquisition and Preparation {RYAN}

## Acqusition

In [ ]:
# Decided to do cats - why

In [ ]:
# Created a scraper to get urls we would use based on a search - came out to 1000
 # Call the scraper

In [ ]:
# Used the acquisition function json to df
 # Modified to take in repos from above process
 # Output is Json, we convert it to dataframe

In [ ]:
# Description of acquired features
 # read_me content
 # top language

## Preparation

In [ ]:
# We gotta fix the readme content - its gibberish

In [ ]:
# Applied the following cleaning steps (what and quick why for each) to the read_me content

In [ ]:
# Applied the cleaner to the df 

# $\bigstar$ Univariate Summary Analysis {RYAN}

In [ ]:
# Languages

In [ ]:
# Run a wrangle function on it to get the splits but also df with language_group

In [ ]:
# Language group histogram

In [ ]:
# Word count histogram and discussion

# $\bigstar$ EDA {RYAN}

In [1]:
# Questions that we have - does word count have an impact?  Hypothesis test mean words and language_group

In [ ]:
# Does number of unique words have an impact?

In [ ]:
# Does unique words as a percentage of total words have an impact?  OR TF-IDF IF IT MAKES SENSE

## Wordclouds and Bigrams {RYAN w/ GREG INPUT}

In [2]:
# We'll pick out what should go into the final from your scrap work

#  $\bigstar$ Modeling

In [ ]:
# Models - Try all classification models.  Try them validate and then test.

#  $\bigstar$ Conclusions

### Key Findings

### Model Effectiveness

### Recommendations and Next Steps